In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import math
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
import scipy.io as sio
import pandas as pd
import time
import cv2

In [3]:
path_data='/content/drive/MyDrive/dataset/dataset_classes/'
folders=glob.glob(path_data+'*')
classes=np.array(['MISC', 'CIRC', 'CALC', 'SPIC', 'ASYM', 'ARCH', 'NORM'])

In [4]:
data_id=[]
for i in folders:
  print(i[i.rfind('/')+1:])
  if i==0:
    data_id=np.array(glob.glob(i+'/m*'))
  else:
    images=np.array(glob.glob(i+'/m*'))
    data_id=np.concatenate((data_id, images))

MISC
CIRC
CALC
SPIC
ASYM
ARCH
NORM


In [5]:
ids=sio.loadmat('/content/drive/MyDrive/dataset/MIAS_kaggle/sub_id.mat')['Id']
dic=sio.loadmat('/content/drive/MyDrive/dataset/MIAS_kaggle/Dic.mat')

In [6]:
def labels(key):
  return np.where(classes==dic[key][0])[0][0]
vecx=np.vectorize(labels)

In [7]:
def remove(strg):
  return strg[47:] 
removv=np.vectorize(remove)

In [8]:
inds_x=removv(data_id)

In [9]:
def get_data(x_id, Fourier=False):
  if Fourier:
    pos=x_id.find('/')+1
    name=x_id[:pos]+'Fourier_'+x_id[pos:]
  else:
    name=x_id
  ima = cv2.imread(path_data+name)
  lab=np.where(classes==x_id[:x_id.find('/')])[0][0]
  return ima, int(lab)

In [10]:
def one_hot(vec):
  vec = vec
  shape = (vec.size, 7)
  one = np.zeros(shape)
  rows = np.arange(vec.size)
  one[rows, vec.astype('int')] = 1
  return one

In [11]:
def get_index(inds_0):
  subx=np.shape(inds_0)[0]
  rando_ind=np.random.choice(subx,subx,replace=False)

  idx=inds_0[rando_ind]

  train_id=idx[:int(subx*0.7)]
  valid_id=idx[int(subx*0.7):int(subx*0.85)]
  test_id =idx[int(subx*0.85):]
  return train_id, valid_id, test_id

In [12]:
def get_index2(inds_0):
  val=vecx(inds_0)
  train_id=[]
  valid_id=[]
  test_id =[]

  for ix in range(7):
    pos=np.where(val==ix)[0]
    sz=np.shape(pos)[0]
    rx=np.random.choice(sz,sz,replace=False)
    pos=pos[rx]
    train_id=np.append(train_id, inds_0[pos[:int(sz*0.6)]])
    valid_id=np.append(valid_id, inds_0[pos[int(sz*0.6):int(sz*0.8)]])
    test_id =np.append(test_id , inds_0[pos[int(sz*0.8):]])

  stn=np.shape(train_id)[0]
  sva=np.shape(valid_id)[0]
  stt=np.shape(test_id)[0]

  train_id=train_id[np.random.choice(stn,stn,replace=False)]
  valid_id=valid_id[np.random.choice(sva,sva,replace=False)]
  test_id =test_id[np.random.choice(stt,stt,replace=False)]
  return train_id, valid_id, test_id

In [13]:
kernel=[(0,0), (0,2), (1,1), (1,3), (2,0), (2,2), (3,1), (3,3)]
def get_ima(tam, pk, ima, x_id):
  ver=np.zeros((tam,tam))
  hor=np.zeros((tam,tam))
  ver[np.arange(tam)%4==kernel[pk][0],:]=1
  hor[:,np.arange(tam)%4==kernel[pk][1]]=1
  ima=ima[...,0]
  ima=ima[ver*hor==1].reshape((int(tam/4),int(tam/4)))
  if kernel[pk][0]>=2:
    ima=np.flip(ima, axis=0)
  ima=np.rot90(ima,kernel[pk][1])
  lab=labels(x_id)
  return ima, int(lab)

In [14]:
def get_data_reduction(x_id, Fourier=False, pk=np.random.choice(8), aug=True):
  tam=1024
  if Fourier:
    pos=x_id.find('/')+1
    name='/content/drive/MyDrive/dataset/MIAS_kaggle/all-mias/'+x_id+'.pgm'
  else:
    name='/content/drive/MyDrive/dataset/MIAS_kaggle/all-mias/'+x_id+'.pgm'
  ima = cv2.imread(name)  
  if np.shape(ima)[0]!=tam:
    print('Size error')
  else:
    if aug:
      imaOu=np.zeros((8,int(tam/4),int(tam/4)))
      lab2=[]
      for pk in range(8):
        im, la=get_ima(tam, pk, ima, x_id)
        imaOu[pk]=im
        lab2=np.append(lab2,la)
      return imaOu, lab2
    else:
      return get_ima(tam, pk, ima, x_id)

In [ ]:
xx, yy=get_data_reduction('mdb013', aug=True)
np.shape(xx), np.shape(yy)

In [ ]:
plt.figure(figsize=(20,20))
for i in range(8):
  plt.subplot(4,4,i+1)
  plt.title(yy[i])
  plt.imshow(xx[i])
  plt.axis('off')

In [ ]:
import tensorflow as tf

In [ ]:
def get_test_data(test_id):
  xx_test=[]
  yx_test=[]
  for n, i in enumerate(test_id):
    uno,dos=get_data(i)
    xx_test.append(uno)
    yx_test.append(dos)
  test_x=np.array(xx_test)
  test_y=np.array(yx_test)
  return test_x, one_hot(test_y)

In [ ]:
def get_matrix(ind_0, aug=False, rgb=True):
  sz=np.shape(ind_0)[0]
  ima,lab=get_data_reduction(ind_0[0], aug=aug)
  szx=np.shape(ima)
  mat=np.zeros((sz*(1+int(aug)*7),szx[0+int(aug)],szx[1+int(aug)]))
  labels=[]

  for n,ix in enumerate(ind_0):
    ima,lab=get_data_reduction(ix, aug=aug)
    mat[n*(1+int(aug)*7):(n+1)*(1+int(aug)*7)]=ima
    labels=np.append(labels, lab)
    print("\rprocess ", round((n+1)*100/sz, 2), "%", end="")
  mat=tf.convert_to_tensor(mat.reshape(np.shape(mat)+(1,)))
  labels=tf.convert_to_tensor(one_hot(labels))
  if rgb:
    return tf.image.grayscale_to_rgb(mat), labels
  else:
    return mat, labels

In [ ]:
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

In [ ]:
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [ ]:
train,valid,test=get_index2(ids)
x_train, y_train=get_matrix(train, True)
x_valid, y_valid=get_matrix(valid, True)
x_test , y_test =get_matrix(test,  True)

In [ ]:
from tensorflow.keras import applications as ap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
Networs=['InceptionV3', 'DenseNet121', 'InceptionResNetV2', 'ResNet50V2', 'VGG19', 'Xception']
def get_model(network, opt='adadelta', loss_name='categorical_crossentropy', input_shape=(256,256,3)):
  try:
    del model
  except:
    print('done')
  model=Sequential()
  if network=='ResNet50V2':
    model.add(ap.ResNet50V2(include_top=False, input_shape=input_shape, pooling='avg', classes=7)) # The input must have 3 channels
  if network=='EfficientNetB7':
    model.add(ap.EfficientNetB7(include_top=False, input_shape=input_shape, pooling='avg', classes=7)) 
  if network=='InceptionResNetV2':
    model.add(ap.InceptionResNetV2(include_top=False, input_shape=input_shape, pooling='avg', classes=7)) 
  if network=='InceptionV3':
    model.add(ap.InceptionV3(include_top=False, input_shape=input_shape, pooling='avg', classes=7)) 
  if network=='NASNetLarge':
    model.add(ap.NASNetLarge(include_top=False, input_shape=input_shape, pooling='avg', classes=7))
  if network=='VGG19':
    model.add(ap.VGG19(include_top=False, input_shape=input_shape, pooling='avg', classes=7)) 
  if network=='Xception':
    model.add(ap.Xception(include_top=False, input_shape=input_shape, pooling='avg', classes=7))
  if network=='DenseNet121':
    model.add(ap.DenseNet121(include_top=False, input_shape=input_shape, pooling='avg', classes=7))
  
  model.add(Dense(7, activation='softmax'))
  model.compile(optimizer=opt, loss=loss_name, metrics=['acc', tf.keras.metrics.Recall(), tf.keras.metrics.FalsePositives()])
  return model

In [ ]:
df = pd.DataFrame(columns=('run_n', 'network', 'optimizer', 'loss', 'epochs', 'total_parameters', 'time', 'augm', 'Class', 'TP', 'TN', 'FP', 'FN','result_mat'))
df.head()

In [ ]:
from IPython.display import clear_output

path2='/content/drive/MyDrive/INDIGO_MLR/CancerDeMama_MIAS/data_augmentation/'
pathW='/content/drive/MyDrive/INDIGO_MLR/CancerDeMama_MIAS/Weights/'
augmentation='partsRotated'
optimizer='adadelta'
epochs=40
loss='categorical_crossentropy'

In [ ]:
TP=tf.keras.metrics.TruePositives()
TN=tf.keras.metrics.TrueNegatives()
FP=tf.keras.metrics.FalsePositives()
FN=tf.keras.metrics.FalseNegatives()

In [ ]:
for net in Networs:

  name_0='_'.join(['clasification_t',augmentation,net,optimizer])
  if not os.path.exists(path2+name_0+'.csv'):
    df.to_csv(path2+name_0+'.csv')

  for i in range(14,80): #Numero corridas 
    for j in [loss]: #Funciones de perdida
      cntn=True
      name_m='_'.join([name_0,j,'run',str(i)])  
      print(name_m)

      model=get_model(net)
      
      try:
        tic = time.time()
        results = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=epochs)
        toc=time.time()-tic
        model.save_weights(pathW+name_m+"w.h5")
      except:
        print('Training error')
        cntn=False

      if cntn:
        sio.savemat(pathW+name_m+'_r.mat', results.history)

        #Validation
        y_hat=np.array(model.predict(x_test))
        sio.savemat(pathW+name_m+'_los_8.mat',{'y_hat': y_hat})

        for class_i in range(7):
          TP.reset_state()
          TN.reset_state()
          FP.reset_state()
          FN.reset_state()        

          TP.update_state(y_hat[:,class_i], y_test[:,class_i])
          TN.update_state(y_hat[:,class_i], y_test[:,class_i])
          FP.update_state(y_hat[:,class_i], y_test[:,class_i])
          FN.update_state(y_hat[:,class_i], y_test[:,class_i])
          total_p=model.count_params()

          #data frame
          df2=pd.read_csv(path2+name_0+'.csv')
          df2=df2.append({'run_n': i,
                        'network': net,
                        'optimizer': optimizer,
                        'loss': 'categorical_crossentropy',
                        'epochs': epochs,
                        'total_parameters': total_p,
                        'time': toc,
                        'augm': augmentation,
                        'Class': class_i,
                        'TP': float(TP.result()),
                        'TN': float(TN.result()),
                        'FP': float(FP.result()),
                        'FN': float(FN.result()),
                        'result_mat': name_m+'_r.mat'} , ignore_index=True)
          df2=df2.drop(df2.columns[:np.where(df2.columns=='run_n')[0][0]], axis=1)
          df2.to_csv(path2+name_0+'.csv')
        clear_output(wait=True)